In [1]:
spark

In [2]:
sc.master

'yarn'

In [3]:
!hdfs dfs -ls

Found 2 items
drwxr-xr-x   - root hadoop          0 2022-04-16 13:30 .sparkStaging
drwxr-xr-x   - root hadoop          0 2022-04-15 06:44 hw3


In [4]:
ls

Youvegottofindwhatyoulove.txt  lib32@       root/
bin@                           lib64@       run/
boot/                          libx32@      sbin@
copyright                      lost+found/  srv/
dev/                           media/       sys/
etc/                           mnt/         tmp/
hadoop/                        opt/         usr/
home/                          output/      var/
lib@                           proc/        yellow_tripdata_2018-10.csv


In [5]:
# !wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-10.csv

In [6]:
!cat yellow_tripdata_2018-10.csv | head -5

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount




cat: write error: Broken pipe


In [7]:
!ls

Youvegottofindwhatyoulove.txt  home	   mnt	   srv
bin			       lib	   opt	   sys
boot			       lib32	   output  tmp
copyright		       lib64	   proc    usr
dev			       libx32	   root    var
etc			       lost+found  run	   yellow_tripdata_2018-10.csv
hadoop			       media	   sbin


In [8]:
!hdfs dfs -put yellow_tripdata_2018-10.csv /hw3/input/

put: `/hw3/input/yellow_tripdata_2018-10.csv': File exists


In [9]:
!hdfs dfs -ls /hw3/input

Found 2 items
-rw-r--r--   2 yes19990817 hadoop      11948 2022-04-15 05:58 /hw3/input/Youvegottofindwhatyoulove.txt
-rw-r--r--   2 root        hadoop  779465756 2022-04-16 10:07 /hw3/input/yellow_tripdata_2018-10.csv


In [21]:
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark.sql.functions as F

conf = SparkConf() \
        .setAppName("Q2") \

sc = SparkContext.getOrCreate();

df = spark.read.csv('/hw3/input/yellow_tripdata_2018-10.csv',sep=',',header=True,inferSchema=True)
df.show(5, False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|1       |2018-10-01 00:23:34 |2018-10-01 00:44:50  |1              |6.2          |1         |N                 |68          |7           |2           |20.5       |0.5  |0.5    |0.0       |0.0         |0.3                  |21.8        |
|1       |2018-10-01 00:40:05 |2018-10-01 01:01:

In [22]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [23]:
from pyspark.sql.functions import desc
dfg=df.groupby('passenger_count').count().sort(desc("count"))
dfg.show(10)

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|6301766|
|              2|1283324|
|              5| 373592|
|              3| 366264|
|              6| 226545|
|              4| 163341|
|              0| 106166|
|              7|     44|
|              8|     34|
|              9|     29|
+---------------+-------+



In [24]:
total= df.count()
total

8821105

In [25]:
#expect passenger_count=0
total-106166

8714939

In [26]:
avg=(6301766*1+1283324*2+373592*5+366264*3+226545*6+163341*4+44*7+34*8+29*9)/8714939
avg

1.5890691833872848

In [27]:
6301766/8821105

0.7143964389948878

In [43]:
# 用1來補passenger count=0

In [53]:
# replace the missing values in passenger_count with 1.
dfnew = df.withColumn("passenger_count", F.when((df["passenger_count"] == "0"), 1).otherwise(df["passenger_count"]))

In [54]:
dfgnew=dfnew.groupby('passenger_count').count().sort(desc("count"))
dfgnew.show(10)

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|6407932|
|              2|1283324|
|              5| 373592|
|              3| 366264|
|              6| 226545|
|              4| 163341|
|              7|     44|
|              8|     34|
|              9|     29|
+---------------+-------+



In [45]:
#credit card
df1 = dfnew.filter(dfnew["payment_type"] == "1").select("passenger_count", "total_amount")
df1.show(5)

+---------------+------------+
|passenger_count|total_amount|
+---------------+------------+
|              1|       25.35|
|              1|       14.75|
|              1|       70.27|
|              1|       20.16|
|              3|        18.8|
+---------------+------------+
only showing top 5 rows



In [46]:
average_amounts1 = df1.groupby('passenger_count').agg({"total_amount": "average"})
average_amounts1.show()

+---------------+------------------+
|passenger_count| avg(total_amount)|
+---------------+------------------+
|              1| 18.09680078351181|
|              6|18.137538831763177|
|              3|18.397956039882864|
|              5| 18.35350646849934|
|              9| 78.46608695652174|
|              4|18.679751424181195|
|              8|             74.32|
|              7| 64.35026315789474|
|              2|18.823159226021037|
+---------------+------------------+



In [47]:
df2 = dfnew.filter(df["payment_type"] == "2") .select("passenger_count", "total_amount")
df2.show(5)

+---------------+------------+
|passenger_count|total_amount|
+---------------+------------+
|              1|        21.8|
|              1|        36.3|
|              1|         8.3|
|              1|        10.3|
|              2|         5.8|
+---------------+------------+
only showing top 5 rows



In [48]:
average_amounts2 = df2.groupby('passenger_count').agg({"total_amount": "average"})
average_amounts2.show(10)

+---------------+------------------+
|passenger_count| avg(total_amount)|
+---------------+------------------+
|              1|13.682974942407997|
|              6|13.792589226502662|
|              3|14.810038897804311|
|              5|13.900753015657697|
|              9|52.521666666666675|
|              4|15.506981138430348|
|              8| 72.89999999999999|
|              7| 53.98666666666667|
|              2| 14.72907295625041|
+---------------+------------------+



In [36]:
# 改用2來補passenger count=0

In [55]:
# replace the missing values in passenger_count with 2.
dfnew = df.withColumn("passenger_count", F.when((df["passenger_count"] == "0"), 2).otherwise(df["passenger_count"]))
dfgnew=dfnew.groupby('passenger_count').count().sort(desc("count"))
dfgnew.show(10)

+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|6301766|
|              2|1389490|
|              5| 373592|
|              3| 366264|
|              6| 226545|
|              4| 163341|
|              7|     44|
|              8|     34|
|              9|     29|
+---------------+-------+



In [56]:
#credit card
df1 = dfnew.filter(dfnew["payment_type"] == "1").select("passenger_count", "total_amount")
average_amounts1 = df1.groupby('passenger_count').agg({"total_amount": "average"})
average_amounts1.show()

+---------------+------------------+
|passenger_count| avg(total_amount)|
+---------------+------------------+
|              1|18.095616294838994|
|              6|18.137538831763177|
|              3|18.397956039882867|
|              5|18.353506468499337|
|              9| 78.46608695652174|
|              4|18.679751424181198|
|              8|             74.32|
|              7| 64.35026315789473|
|              2|18.772382505987768|
+---------------+------------------+



In [57]:
df2 = dfnew.filter(df["payment_type"] == "2") .select("passenger_count", "total_amount")
average_amounts2 = df2.groupby('passenger_count').agg({"total_amount": "average"})
average_amounts2.show(10)

+---------------+------------------+
|passenger_count| avg(total_amount)|
+---------------+------------------+
|              1|13.682682121839113|
|              6| 13.79258922650266|
|              3|14.810038897804311|
|              5|13.900753015657697|
|              9|52.521666666666675|
|              4|15.506981138430348|
|              8| 72.89999999999999|
|              7| 53.98666666666666|
|              2|14.657005030646406|
+---------------+------------------+

